In [24]:
import os
import itertools

In [25]:
def mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [26]:
# global job parameters

job_directory = f"object_sorting"
out_dir = f'{job_directory}/.out'
time_str = '00-08:00:00'
partition = 'gpu'
ntasks = 1
nodes = 1
cpu_per_task = 8
mem_per_cpu = 2
n_gpus = 1
gpus_constraints = '"a100|rtx3090|v100|rtx2080ti"' # all gpus are pretty good now
project_dir = "/gpfs/gibbs/project/lafferty/ma2393/abstract_transformer/experiments/object_sorting"

mkdir(job_directory)
mkdir(out_dir)

In [27]:
# define params of individual jobs

jobs_params = [
    dict(ee=2, ea=2, de=2, da=2),
    dict(ee=2, ea=2, de=2, da=0),
    dict(ee=0, ea=2, de=2, da=0),
    dict(ee=0, ea=4, de=4, da=0),
    dict(ee=2, ea=0, de=2, da=0),
    dict(ee=4, ea=0, de=4, da=0),
]

In [28]:
jobs_params

[{'ee': 2, 'ea': 2, 'de': 2, 'da': 2},
 {'ee': 2, 'ea': 2, 'de': 2, 'da': 0},
 {'ee': 0, 'ea': 2, 'de': 2, 'da': 0},
 {'ee': 0, 'ea': 4, 'de': 4, 'da': 0},
 {'ee': 2, 'ea': 0, 'de': 2, 'da': 0},
 {'ee': 4, 'ea': 0, 'de': 4, 'da': 0}]

In [29]:
# create jobs
created_jobs = []
for params in jobs_params:

    job_name = f"object_sorting-ee{params['ee']}-ea{params['ea']}-de{params['de']}-da{params['da']}"

    job_file = os.path.join(job_directory, f"{job_name}.job")

    with open(job_file, 'w') as fh:
        fh.writelines(f"#!/bin/bash\n")
        fh.writelines(f"#SBATCH --partition={partition}\n")
        fh.writelines(f"#SBATCH --job-name={job_name}\n")
        fh.writelines(f"#SBATCH --output={out_dir}/%j-{job_name}.out\n")
        fh.writelines(f"#SBATCH --ntasks={ntasks} --nodes={nodes}\n")
        fh.writelines(f"#SBATCH --cpus-per-task={cpu_per_task}\n")
        fh.writelines(f"#SBATCH --mem-per-cpu={mem_per_cpu}G\n")
        fh.writelines(f"#SBATCH --time={time_str}\n")
        fh.writelines(f"#SBATCH --mail-type=ALL\n")
        fh.writelines(f"#SBATCH --gpus={n_gpus}\n")
        fh.writelines(f"#SBATCH -C {gpus_constraints}\n")# --gpus={n_gpus}\n")

        fh.writelines('\n')
        fh.writelines('module load StdEnv\n')
        fh.writelines('export SLURM_EXPORT_ENV=ALL\n')
        fh.writelines('\n')

        # fh.writelines(f"module restore python_env\n") # load modules i need
        fh.writelines(f"module load miniconda\n") # load modules i need
        # fh.writelines(f"conda init\n")
        fh.writelines(f"conda activate abstract_transformer\n") # activate conda environment
        fh.writelines(f"conda info --envs\n") # activate conda environment

        fh.writelines('\n')
        fh.writelines(f"nvidia-smi -L\n") # print gpu information
        fh.writelines('\n')

        fh.writelines(f"cd {project_dir}\n") # navigate to project directory
        # run python script
        fh.writelines(f"python eval_learning_curves.py --ee {params['ee']} --ea {params['ea']} --de {params['de']} --da {params['da']}\n")

    created_jobs.append(job_file)

In [30]:
created_jobs

['object_sorting/object_sorting-ee2-ea2-de2-da2.job',
 'object_sorting/object_sorting-ee2-ea2-de2-da0.job',
 'object_sorting/object_sorting-ee0-ea2-de2-da0.job',
 'object_sorting/object_sorting-ee0-ea4-de4-da0.job',
 'object_sorting/object_sorting-ee2-ea0-de2-da0.job',
 'object_sorting/object_sorting-ee4-ea0-de4-da0.job']